In [4]:
import pandas as pd, requests

In [5]:
response = requests.get('http://api.open-notify.org/astros.json')
print(response.status_code)

200


In [6]:
# Return the raw bytes of the data payload
# response.content()

# Return a string representation of the data payload
# response.text()

# This method is convenient when the API returns JSON
data = response.json()

In [ ]:
type(response.json())

dict

# Data Processing

## Looping

In [7]:
data

{'people': [{'name': 'Mark Vande Hei', 'craft': 'ISS'},
  {'name': 'Oleg Novitskiy', 'craft': 'ISS'},
  {'name': 'Pyotr Dubrov', 'craft': 'ISS'},
  {'name': 'Thomas Pesquet', 'craft': 'ISS'},
  {'name': 'Megan McArthur', 'craft': 'ISS'},
  {'name': 'Shane Kimbrough', 'craft': 'ISS'},
  {'name': 'Akihiko Hoshide', 'craft': 'ISS'},
  {'name': 'Nie Haisheng', 'craft': 'Tiangong'},
  {'name': 'Liu Boming', 'craft': 'Tiangong'},
  {'name': 'Tang Hongbo', 'craft': 'Tiangong'}],
 'number': 10,
 'message': 'success'}

In [ ]:
message = data['message']
number = data['number']
people = data['people']

processed_data = []
for person in people:
    craft = person['craft']
    name = person['name']
    processed_data.append([message, number, craft, name])

print(processed_data)

[['success', 7, 'ISS', 'Sergey Ryzhikov'], ['success', 7, 'ISS', 'Kate Rubins'], ['success', 7, 'ISS', 'Sergey Kud-Sverchkov'], ['success', 7, 'ISS', 'Mike Hopkins'], ['success', 7, 'ISS', 'Victor Glover'], ['success', 7, 'ISS', 'Shannon Walker'], ['success', 7, 'ISS', 'Soichi Noguchi']]


In [ ]:
df1 = pd.DataFrame(processed_data, columns=['message', 'number', 'craft', 'name'])
df1

,message,number,craft,name
0,success,7,ISS,Sergey Ryzhikov
1,success,7,ISS,Kate Rubins
2,success,7,ISS,Sergey Kud-Sverchkov
3,success,7,ISS,Mike Hopkins
4,success,7,ISS,Victor Glover
5,success,7,ISS,Shannon Walker
6,success,7,ISS,Soichi Noguchi


## Pandas Cross Join

In [ ]:
df2 = pd.DataFrame(data)
df2

,message,number,people
0,success,7,"{'craft': 'ISS', 'name': 'Sergey Ryzhikov'}"
1,success,7,"{'craft': 'ISS', 'name': 'Kate Rubins'}"
2,success,7,"{'craft': 'ISS', 'name': 'Sergey Kud-Sverchkov'}"
3,success,7,"{'craft': 'ISS', 'name': 'Mike Hopkins'}"
4,success,7,"{'craft': 'ISS', 'name': 'Victor Glover'}"
5,success,7,"{'craft': 'ISS', 'name': 'Shannon Walker'}"
6,success,7,"{'craft': 'ISS', 'name': 'Soichi Noguchi'}"


In [ ]:
tb1 = df2[['message', 'number']]
tb1 = tb1.drop_duplicates()
tb1

,message,number
0,success,7


In [ ]:
tb2 = pd.DataFrame(data['people'], columns=['craft', 'name'])
tb2

,craft,name
0,ISS,Sergey Ryzhikov
1,ISS,Kate Rubins
2,ISS,Sergey Kud-Sverchkov
3,ISS,Mike Hopkins
4,ISS,Victor Glover
5,ISS,Shannon Walker
6,ISS,Soichi Noguchi


In [ ]:
# Now to perform cross join, we will create a key column in both the DataFrames to merge on that key. 
tb1['key'] = 1
tb2['key'] = 1

In [ ]:
# Perform Cross Join
df2 = pd.merge(tb1, tb2, on ='key').drop("key", 1) 
df2

,message,number,craft,name
0,success,7,ISS,Sergey Ryzhikov
1,success,7,ISS,Kate Rubins
2,success,7,ISS,Sergey Kud-Sverchkov
3,success,7,ISS,Mike Hopkins
4,success,7,ISS,Victor Glover
5,success,7,ISS,Shannon Walker
6,success,7,ISS,Soichi Noguchi


# Alphavantage

In [ ]:
queries = {'apikey': 'XVV7I3GHUW42LTO4',
            'function': 'TIME_SERIES_INTRADAY',
            'symbol': 'IBM',
            'interval': '5min'}
response = requests.get('https://www.alphavantage.co/query?', params=queries,)
response.status_code

200

In [ ]:
json_data = response.json()
json_data

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Symbol': 'IBM',
  '3. Last Refreshed': '2021-02-19 18:45:00',
  '4. Interval': '5min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2021-02-19 09:10:00': {'1. open': '121.0500',
   '2. high': '121.0500',
   '3. low': '121.0500',
   '4. close': '121.0500',
   '5. volume': '109'},
  '2021-02-19 09:15:00': {'1. open': '121.1000',
   '2. high': '121.1000',
   '3. low': '121.0500',
   '4. close': '121.0500',
   '5. volume': '878'},
  '2021-02-19 09:20:00': {'1. open': '121.0800',
   '2. high': '121.1300',
   '3. low': '121.0800',
   '4. close': '121.1200',
   '5. volume': '4871'},
  '2021-02-19 09:25:00': {'1. open': '121.0500',
   '2. high': '121.1300',
   '3. low': '121.0500',
   '4. close': '121.1300',
   '5. volume': '1628'},
  '2021-02-19 09:30:00': {'1. open': '121.0700',
   '2. high': '121.1100',
   '3. low': '120.8000',
   '4. close': '120.80

In [ ]:
symbol = json_data['Meta Data']['2. Symbol']
interval = json_data['Meta Data']['4. Interval']
base = pd.DataFrame({'symbol': [symbol], 'interval': [interval], 'key': [1]})

In [ ]:
prices = json_data['Time Series (5min)']

In [ ]:
prices = pd.DataFrame(prices).T
prices.head()

,1. open,2. high,3. low,4. close,5. volume
2021-02-19 18:45:00,119.1800,119.1800,119.1800,119.1800,1250
2021-02-19 18:25:00,119.1000,119.1000,119.1000,119.1000,701
2021-02-19 18:20:00,119.0000,119.0000,119.0000,119.0000,490
2021-02-19 18:05:00,119.1500,119.1500,119.1500,119.1500,353
2021-02-19 18:00:00,119.0500,119.0500,119.0500,119.0500,556


In [ ]:
#prices.reset_index(inplace=True)
prices['key'] = 1
prices.head()

,index,1. open,2. high,3. low,4. close,5. volume,key
0,2021-02-19 18:45:00,119.1800,119.1800,119.1800,119.1800,1250,1
1,2021-02-19 18:25:00,119.1000,119.1000,119.1000,119.1000,701,1
2,2021-02-19 18:20:00,119.0000,119.0000,119.0000,119.0000,490,1
3,2021-02-19 18:05:00,119.1500,119.1500,119.1500,119.1500,353,1
4,2021-02-19 18:00:00,119.0500,119.0500,119.0500,119.0500,556,1


In [ ]:
result = pd.merge(base, prices, on ='key').drop("key", 1)
result.head()

,symbol,interval,index,1. open,2. high,3. low,4. close,5. volume
0,IBM,5min,2021-02-19 18:45:00,119.1800,119.1800,119.1800,119.1800,1250
1,IBM,5min,2021-02-19 18:25:00,119.1000,119.1000,119.1000,119.1000,701
2,IBM,5min,2021-02-19 18:20:00,119.0000,119.0000,119.0000,119.0000,490
3,IBM,5min,2021-02-19 18:05:00,119.1500,119.1500,119.1500,119.1500,353
4,IBM,5min,2021-02-19 18:00:00,119.0500,119.0500,119.0500,119.0500,556


In [ ]:
result.columns = ['symbol', 'interval', 'date', 'open', 'high', 'low', 'close', 'volume']
result.head()

,symbol,interval,date,open,high,low,close,volume
0,IBM,5min,2021-02-19 18:45:00,119.1800,119.1800,119.1800,119.1800,1250
1,IBM,5min,2021-02-19 18:25:00,119.1000,119.1000,119.1000,119.1000,701
2,IBM,5min,2021-02-19 18:20:00,119.0000,119.0000,119.0000,119.0000,490
3,IBM,5min,2021-02-19 18:05:00,119.1500,119.1500,119.1500,119.1500,353
4,IBM,5min,2021-02-19 18:00:00,119.0500,119.0500,119.0500,119.0500,556
